In [1]:
from google.colab import files
uploaded = files.upload()

Saving co2.csv to co2.csv


In [2]:
# Import thư viện cần thiết
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score


In [3]:
# Hàm tạo các cột trễ (lag features) và cột target cho bài toán dự báo
def create_timeseries_data(data, window_size, target_size):
    # Tạo các feature từ các giá trị CO2 trước đó
    i = 1
    while i < window_size:
        data[f"co2_{i}"] = data["co2"].shift(-i)
        i += 1

    # Tạo cột target cho các bước dự báo
    i = 0
    while i < target_size:
        data[f"target_{i}"] = data["co2"].shift(-i - window_size)
        i += 1

    # Loại bỏ các hàng bị NaN do shift
    data = data.dropna(axis=0)
    return data


In [4]:
# Đọc file CSV và xử lý dữ liệu ban đầu
data = pd.read_csv("co2.csv")
data["time"] = pd.to_datetime(data["time"], yearfirst=True)

# Xử lý missing values bằng nội suy
data["co2"] = data["co2"].interpolate()

# Tạo dữ liệu time series
window_size = 10  # số bước đầu vào
target_size = 5   # số bước dự báo
train_ratio = 0.8

data = create_timeseries_data(data, window_size, target_size)

# Xác định biến X và y
targets = [f"target_{i}" for i in range(target_size)]
x = data.drop(["time"] + targets, axis=1)
y = data[targets]

# Chia train/test thủ công (time series nên không shuffle)
num_samples = len(x)
x_train = x[:int(num_samples * train_ratio)]
y_train = y[:int(num_samples * train_ratio)]
x_test = x[int(num_samples * train_ratio):]
y_test = y[int(num_samples * train_ratio):]

print(f"Số lượng mẫu train: {len(x_train)}, test: {len(x_test)}")


Số lượng mẫu train: 1816, test: 454


In [5]:
# Train mô hình Linear Regression cho từng bước dự báo
models = [LinearRegression() for _ in range(target_size)]

for idx, model in enumerate(models):
    model.fit(x_train, y_train[f"target_{idx}"])
    y_predict = model.predict(x_test)
    print(f"===== Model dự báo bước {idx+1} =====")
    print(f"MAE: {mean_absolute_error(y_test[f'target_{idx}'], y_predict):.4f}")
    print(f"MSE: {mean_squared_error(y_test[f'target_{idx}'], y_predict):.4f}")
    print(f"RMSE: {root_mean_squared_error(y_test[f'target_{idx}'], y_predict):.4f}")
    print(f"R2: {r2_score(y_test[f'target_{idx}'], y_predict):.4f}")
    print("-" * 50)


===== Model dự báo bước 1 =====
MAE: 0.3604
MSE: 0.2177
RMSE: 0.4665
R2: 0.9908
--------------------------------------------------
===== Model dự báo bước 2 =====
MAE: 0.4964
MSE: 0.4093
RMSE: 0.6397
R2: 0.9827
--------------------------------------------------
===== Model dự báo bước 3 =====
MAE: 0.6149
MSE: 0.6105
RMSE: 0.7813
R2: 0.9743
--------------------------------------------------
===== Model dự báo bước 4 =====
MAE: 0.7762
MSE: 0.9274
RMSE: 0.9630
R2: 0.9610
--------------------------------------------------
===== Model dự báo bước 5 =====
MAE: 0.9386
MSE: 1.3644
RMSE: 1.1681
R2: 0.9427
--------------------------------------------------
